In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [173]:
vod = pd.read_csv('../data/vod89.csv')
vod

,subsr,program,main_cat,sub_cat,score,category,rename
0,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날
1,59900000,그것이알고싶다,TV 시사/교양,기타,0.374741,TV프로그램,그것이알고싶다
2,59900000,마파도,영화,코미디,0.292893,영화,마파도
3,59900000,범죄도시3,영화,액션/어드벤쳐,0.292893,영화,범죄도시3
4,59900000,초대: 스와핑 데이,영화,멜로,0.292893,영화,초대:스와핑데이
...,...,...,...,...,...,...,...
1040,67055000,궁금한이야기Y,TV 연예/오락,기타,0.132597,TV프로그램,궁금한이야기Y
1041,67055000,TV소설 은희,TV드라마,기타,0.952381,TV프로그램,TV소설은희
1042,67055000,TV동물농장,TV 연예/오락,기타,0.100000,TV프로그램,TV동물농장
1043,67055000,도망쳐,TV 시사/교양,기타,0.292893,TV프로그램,도망쳐


In [174]:
vod_id = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
vod_id = vod_id[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
5019,일루셔니스트,영화,5019
5020,"파리,13구",영화,5020
5021,미쓰백,영화,5021
5022,그녀는거짓말을너무사랑해,영화,5022


In [175]:
vod1 = vod.merge(vod_id, on = ['rename', 'category'], how = 'left')
userid = pd.DataFrame(vod1['subsr'].unique())
userid.columns = ['subsr']
userid['subsr'] = userid.index

vod1 = vod1.merge(userid, on = ['subsr'], how = 'left')
vod1

,subsr,program,main_cat,sub_cat,score,category,rename,vod_id
0,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날,572
1,59900000,그것이알고싶다,TV 시사/교양,기타,0.374741,TV프로그램,그것이알고싶다,297
2,59900000,마파도,영화,코미디,0.292893,영화,마파도,149
3,59900000,범죄도시3,영화,액션/어드벤쳐,0.292893,영화,범죄도시3,2539
4,59900000,초대: 스와핑 데이,영화,멜로,0.292893,영화,초대:스와핑데이,3316
...,...,...,...,...,...,...,...,...
1040,67055000,궁금한이야기Y,TV 연예/오락,기타,0.132597,TV프로그램,궁금한이야기Y,13
1041,67055000,TV소설 은희,TV드라마,기타,0.952381,TV프로그램,TV소설은희,315
1042,67055000,TV동물농장,TV 연예/오락,기타,0.100000,TV프로그램,TV동물농장,16
1043,67055000,도망쳐,TV 시사/교양,기타,0.292893,TV프로그램,도망쳐,341


In [176]:
vod1.to_csv('../data/vod_score.csv', index = 0)
vod_score = pd.read_csv('../data/vod_score.csv')
vod_score = vod_score[['subsr', 'vod_id', 'score']]
vod_score

,subsr,vod_id,score
0,59900000,572,0.215482
1,59900000,297,0.374741
2,59900000,149,0.292893
3,59900000,2539,0.292893
4,59900000,3316,0.292893
...,...,...,...
1040,67055000,13,0.132597
1041,67055000,315,0.952381
1042,67055000,16,0.100000
1043,67055000,341,0.292893


In [348]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 범위가 0~1 인 경우
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(vod_score)

### KNNBaseline

In [306]:
sim_options = {'name': 'cosine',
               'user_based': True  # user-based similarity
               }
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [307]:
user_id = sorted(vod_score.subsr.unique())
vod_id = sorted(vod_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59900000,0,0.207038
1,59900000,3,0.332038
2,59900000,6,0.268357
3,59900000,7,0.345905
4,59900000,8,0.110351
...,...,...,...
53545,67055000,4629,0.307729
53546,67055000,4634,0.310773
53547,67055000,4647,0.308154
53548,67055000,4698,0.308203


In [241]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59900000,848,858,720,315,1234,824,112,2093,1166,...,1313,1207,836,124,398,323,405,1156,516,751
1,59930000,1234,796,728,757,789,356,754,1077,114,...,751,888,886,1156,1122,111,14,1233,256,1347
2,60067000,954,357,976,731,117,849,1086,992,1639,...,1135,987,490,303,2404,1347,810,1131,1362,2979
3,60224000,1234,357,731,958,796,2457,117,1347,757,...,3502,1233,111,987,398,303,490,381,405,778
4,60326000,858,720,357,849,824,954,117,976,731,...,1070,2357,884,1135,987,490,1347,303,2404,810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,66873000,848,315,724,112,1506,796,2093,2457,414,...,1069,1168,706,955,249,1313,1207,1156,836,124
71,66875000,858,954,720,824,357,976,731,958,796,...,241,114,949,410,992,1639,1523,1069,1068,2775
72,66900000,848,315,112,2093,15,530,450,12,1207,...,527,111,832,3929,1234,1233,1347,858,720,724
73,67008000,1130,1156,47,1234,111,14,1233,1347,848,...,398,357,726,976,2227,731,958,1506,1018,247


In [242]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59900000,"[848, 858, 720, 315, 1234, 824, 112, 2093, 116..."
1,59930000,"[1234, 796, 728, 757, 789, 356, 754, 1077, 114..."
2,60067000,"[954, 357, 976, 731, 117, 849, 1086, 992, 1639..."
3,60224000,"[1234, 357, 731, 958, 796, 2457, 117, 1347, 75..."
4,60326000,"[858, 720, 357, 849, 824, 954, 117, 976, 731, ..."
...,...,...
70,66873000,"[848, 315, 724, 112, 1506, 796, 2093, 2457, 41..."
71,66875000,"[858, 954, 720, 824, 357, 976, 731, 958, 796, ..."
72,66900000,"[848, 315, 112, 2093, 15, 530, 450, 12, 1207, ..."
73,67008000,"[1130, 1156, 47, 1234, 111, 14, 1233, 1347, 84..."


In [243]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = vod_score[vod_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue
    
    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59900000,"[848, 858, 720, 315, 1234, 824, 112, 2093, 116..."
1,59930000,"[1234, 796, 728, 757, 789, 356, 754, 1077, 114..."
2,60067000,"[954, 357, 976, 731, 117, 849, 1086, 992, 1639..."
3,60224000,"[1234, 357, 731, 958, 796, 2457, 117, 1347, 75..."
4,60326000,"[858, 720, 357, 849, 824, 954, 117, 976, 731, ..."
...,...,...
70,66873000,"[848, 315, 724, 112, 1506, 796, 2093, 2457, 41..."
71,66875000,"[858, 954, 720, 824, 357, 976, 731, 958, 796, ..."
72,66900000,"[848, 315, 112, 2093, 15, 530, 450, 12, 1207, ..."
73,67008000,"[1130, 1156, 47, 1234, 111, 14, 1233, 1347, 84..."


In [244]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [245]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp
        
    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.025806451612903233

### SVD, SVDpp

In [399]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(5, 100, 5), 'lr_all': [0.002, 0.005],
              'reg_all': [0.04, 0.06], 'random_state' : [42]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.
              
gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(vod_score[['subsr', 'vod_id', 'score']], reader)

gs.fit(train_set)
gs.best_params

{'mae': {'n_epochs': 95, 'lr_all': 0.005, 'reg_all': 0.06, 'random_state': 42}}

In [505]:
# n_epochs=95, lr_all=0.005, reg_all=0.06, random_state=42 -> 0.045
svd = SVD(n_epochs=200, lr_all=0.003, reg_all=0.06, random_state=42)
svd.fit(trainset)

In [506]:
user_id = sorted(vod_score.subsr.unique())
vod_id = sorted(vod_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svd.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59900000,0,0.273344
1,59900000,3,0.278700
2,59900000,6,0.345904
3,59900000,7,0.389968
4,59900000,8,0.187548
...,...,...,...
53545,67055000,4629,0.229877
53546,67055000,4634,0.185821
53547,67055000,4647,0.276390
53548,67055000,4698,0.149482


In [507]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59900000,991,720,61,884,848,1234,858,1156,724,...,1063,1908,674,1347,2245,414,2093,249,2775,1506
1,59930000,728,796,1156,1233,1506,866,450,858,1234,...,29,112,706,1543,1164,691,1172,1207,516,731
2,60067000,954,720,976,724,849,315,726,992,117,...,1018,754,706,701,124,1506,789,3457,2775,1233
3,60224000,112,858,410,398,888,1168,1639,1233,1156,...,516,2357,1234,1523,1395,498,724,111,954,706
4,60326000,858,1233,720,1086,824,955,1347,315,1639,...,754,954,908,1234,1130,29,498,1135,2357,2457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,66873000,1156,858,720,754,731,976,835,1395,955,...,888,356,2457,1018,450,2093,112,991,724,848
71,66875000,357,731,720,1068,117,858,1156,112,1347,...,2093,516,356,1506,1168,726,754,796,1086,728
72,66900000,848,1156,1234,724,15,2457,1395,810,991,...,1313,1077,2775,14,720,976,816,241,1347,1130
73,67008000,1234,848,1069,731,796,858,476,754,720,...,357,15,1347,81,2495,1018,908,1699,866,4382


In [508]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59900000,"[991, 720, 61, 884, 848, 1234, 858, 1156, 724,..."
1,59930000,"[728, 796, 1156, 1233, 1506, 866, 450, 858, 12..."
2,60067000,"[954, 720, 976, 724, 849, 315, 726, 992, 117, ..."
3,60224000,"[112, 858, 410, 398, 888, 1168, 1639, 1233, 11..."
4,60326000,"[858, 1233, 720, 1086, 824, 955, 1347, 315, 16..."
...,...,...
70,66873000,"[1156, 858, 720, 754, 731, 976, 835, 1395, 955..."
71,66875000,"[357, 731, 720, 1068, 117, 858, 1156, 112, 134..."
72,66900000,"[848, 1156, 1234, 724, 15, 2457, 1395, 810, 99..."
73,67008000,"[1234, 848, 1069, 731, 796, 858, 476, 754, 720..."


In [509]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = vod_score[vod_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue
    
    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59900000,"[991, 720, 61, 884, 848, 1234, 858, 1156, 724,..."
1,59930000,"[728, 796, 1156, 1233, 1506, 866, 450, 858, 12..."
2,60067000,"[954, 720, 976, 724, 849, 315, 726, 992, 117, ..."
3,60224000,"[112, 858, 410, 398, 888, 1168, 1639, 1233, 11..."
4,60326000,"[858, 1233, 720, 1086, 824, 955, 1347, 315, 16..."
...,...,...
70,66873000,"[1156, 858, 720, 754, 731, 976, 835, 1395, 955..."
71,66875000,"[357, 731, 720, 1068, 117, 858, 1156, 112, 134..."
72,66900000,"[848, 1156, 1234, 724, 15, 2457, 1395, 810, 99..."
73,67008000,"[1234, 848, 1069, 731, 796, 858, 476, 754, 720..."


In [510]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [511]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp
        
    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.045161290322580656